<a href="https://colab.research.google.com/github/GraceW18/image-classification-model-CNN/blob/main/RandomizedSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Load and preprocess data
train_data = pd.read_csv('sign_mnist_13bal_train.csv')
test_data = pd.read_csv('sign_mnist_13bal_test.csv')
X_train = train_data.drop('class', axis=1).values / 255.0
y_train = train_data['class'].values
X_test = test_data.drop('class', axis=1).values / 255.0
y_test = test_data['class'].values

# Reshape as images for CNN input (28x28 grayscale)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
num_classes = len(np.unique(y_train))

# Model building function
def build_cnn(optimizer='adam', dropout_rate=0.5, filters=32, dense_units=128):
    model = Sequential()
    model.add(Conv2D(filters, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(2 * filters, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrap with scikeras's KerasClassifier
clf = KerasClassifier (
    model=build_cnn,
    epochs=15,
    batch_size=64,
    verbose=0 # set to 1 for training progress
)

# Define search space
param_dist = {
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.3, 0.5],
    'filters': [32, 64],
    'dense_units': [64, 128],
    'epochs': [10, 15, 20],
    'batch_size': [32, 64, 128]
}

# Set up and run RandomizedSearchCV
random_search = RandomizedSearchCV (
    estimator=clf,
    param_distributions=param_dist,
    n_iter=15, # Number of random combos to try
    scoring='accuracy',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1 # Set to -1 if using CPU server with enough RAM
)
random_search.fit(X_train, y_train)







